<a href="https://colab.research.google.com/github/HelenaNascimento/POS/blob/master/Atividade_ETL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import folium
import folium.plugins

In [ ]:
"""
importar dados do arquivo para o programa
Dados sobre Cidades do Estado do Ceará
Código do IBGE;
Descrição da Cidade;
Posição Longitude e;
Posição Latitude
"""
def load_data_cidades():
    imp_dados_cidade = pd.read_csv('/content/sample_data/localizacao_cidades_CE.csv')
    return imp_dados_cidade


In [ ]:
"""
importar dados sobre vendas dos clientes do mês de janeiro de 2024
Para selecionar a lista usei o select no banco SQL SERVER
  Select top 1000
    substring(cl.Fantasia, 1, 5) as Fantasia,
    Cod_CidIbge,
    sum(ve.Qtd_Total) as Qtd_Total_JAN,
    sum(ve.vlr_total) as vlr_total_JAN
  FROM tab_vendas vd
      inner join tab_cliente cl on vd.cod_clien = cod_clien
  where
    dat_venda >= '20240101'
    dat_venda <= '20240130'
  group by
    Cod_CidIbge

Como resultado da consulta vinheram os 1000 clientes que compraram no mês de Janeiro de 2024:
Fantasia, Código IBGE, Qauntidade total de itens e valor total das compras dentro do perído

"""

def load_data_vendas():
    imp_dados_vendas = pd.read_csv('/content/sample_data/Dados_Vendas_JAN.csv')
    return imp_dados_vendas


In [ ]:

#Após ter carregado os arquivos, agora eu preciso juntar os dados para criar o mapa.


def merge_data(imp_dados_vendas, imp_dados_cidade):
    juncao_dos_dados = pd.merge(imp_dados_vendas, imp_dados_cidade, left_on='Cod_CidIbge', right_on='Cod_IBGE', how='inner') #merge para mapear as informações e fazer a relação entre os arquivos
    filtro_dados = juncao_dos_dados[['Fantasia', 'Cidade', 'Longitude', 'Latitude', 'Qtd_venda', 'Vlr_Venda']]
    return filtro_dados


In [ ]:
#Agora preciso informar onde estão esses clientes no mapa
#É aqui onde a biblioteca folium entra

def create_map(dados):
    mapa = folium.Map(location=[-3.71722, -38.5433], zoom_start=7)  # Coordenadas aproximadas do Ceará
    for _, row in dados.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['Fantasia']}<br>Qtd Venda: {row['Qtd_venda']}<br>Vlr Venda: {row['Vlr_Venda']}",
            tooltip=row['Cidade']
        ).add_to(mapa)
    return mapa

In [ ]:
imp_dados_vendas = load_data_cidades() #Carregar os dados da tabela cidade
imp_dados_cidade = load_data_vendas() #Carregar os dados de Vendas

KeyError: 'Cod_IBGE'

In [ ]:
dados_combinados = merge_data(imp_dados_vendas, imp_dados_cidade) #carregar os dados combinados

In [ ]:
map = create_map(dados_combinados) #criar o mapa

In [ ]:
mapa_vendas